In [1]:
# comment out to run on drive + colab 

# ! pip install transformers_interpret

In [2]:
# from google.colab import drive
# 
# drive.mount('/content/drive/')

In [3]:
import pandas as pd
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers_interpret import MultiLabelClassificationExplainer
import torch
from transformers import pipeline
from datasets import load_from_disk

import warnings
warnings.filterwarnings('ignore')

In [4]:
# comment out the filepath to run on drive + colab 
filepath = "../models/scenario_level/anomic_wernicke"
# filepath = "/content/drive/Shareddrives/AphasiaProject/models/scenario_level/anomic_wernicke"

model = AutoModelForSequenceClassification.from_pretrained(filepath+"/model")
tokenizer = AutoTokenizer.from_pretrained(filepath+"/tokenizer", padding=True, 
                                          truncation=True, return_tensors="pt", 
                                          add_special_tokens=True, max_length=512)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if device == torch.device("cuda"):
    model.to(device)
pipe = pipeline("text-classification", model=filepath+"/model", tokenizer=filepath+"/tokenizer", device=device, truncation=True, padding=True)
cls_explainer = MultiLabelClassificationExplainer(model, tokenizer)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
model.config

DistilBertConfig {
  "_name_or_path": "../models/anomic_wernicke/model",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "ANOMIC",
    "1": "WERNICKE"
  },
  "initializer_range": 0.02,
  "label2id": {
    "ANOMIC": 0,
    "WERNICKE": 1
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "problem_type": "single_label_classification",
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_version": "4.39.0.dev0",
  "vocab_size": 30524
}

In [6]:
model.config.id2label

{0: 'ANOMIC', 1: 'WERNICKE'}

In [7]:
def interpret(data):
    for text, label in zip(list(data["preprocessed_text"]), list(data["label"])):
        if device == torch.device("cuda"):
            inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt", add_special_tokens=True, max_length=512).input_ids.to(device) 
            # tranformers interpret gives error for input of 512 tokens
            if len(inputs[0]) != 512:
                with torch.no_grad():
                    logits = model(inputs).logits
                    
                predicted_class_id = logits.argmax().item()
                print("True: ", label, "Pred: ", model.config.id2label[predicted_class_id])
                print("Text: ", text)
                cls_explainer(text)
                cls_explainer.visualize()
                print("----------------------------------------------------------------------")
        else:
            inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt", add_special_tokens=True, max_length=512) 
            # tranformers interpret gives error for input of 512 tokens
            if len(inputs[0]) != 512:    
                with torch.no_grad():
                    logits = model(**inputs).logits
                    
                predicted_class_id = logits.argmax().item()
                print("True: ", label, "Pred: ", model.config.id2label[predicted_class_id])
                print("Text: ", text)
                cls_explainer(text)
                cls_explainer.visualize()
                print("----------------------------------------------------------------------")

In [8]:
scenarios = ["Speech", "Important_Event", "Cinderella", "Stroke", "Cat"]
# make sure not to interpret on trained scenarios 
texts = load_from_disk(filepath + "/dataset")["test"].remove_columns(["Unnamed: 0", "input_ids", "attention_mask"]).to_pandas()
texts["label"] = [model.config.id2label[x] for x in texts["label"]]
n = 3

# Speech scenario interpretation
Three examples for conduction and three for control (# of examples defined by n)

In [9]:
data = pd.DataFrame(columns=['source_file','scenario','preprocessed_text','label'])

for x in model.config.label2id.keys():
    data = pd.concat([data, texts[(texts["label"] == x) & (texts["scenario"] == scenarios[0])].sample(n=n)])

In [10]:
interpret(data)

True:  ANOMIC Pred:  ANOMIC
Text:  and and [FP] i also think that when i get stuck i [FP] [UP] get help from people. [FP] and they'll fill in or whatever. [FP] and i i can also if if i'm stuck with an idea like i can i use a lot of hand [FP] signal things.


----------------------------------------------------------------------
True:  ANOMIC Pred:  ANOMIC
Text:  hindi.


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
n/a,(0.65),ANOMIC,1.26,[CLS] hindi . [SEP]
n/a,(0.32),WERNICKE,-1.35,[CLS] hindi . [SEP]


----------------------------------------------------------------------
True:  ANOMIC Pred:  ANOMIC
Text:  [FP] it's fine. it's much better than i was. you know i can speak. before i did not. no. no not for two or three years. i have a good speech therapist. [FP] her name is jeanie lastname.


----------------------------------------------------------------------
True:  WERNICKE Pred:  ANOMIC
Text:  uhhuh [FP] i i'm okay.


...

----------------------------------------------------------------------
True:  WERNICKE Pred:  ANOMIC
Text:  very good. yes. it's always different. [FP] the talking they don't work. yeah.


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
n/a,(0.48),ANOMIC,-2.85,[CLS] very good . yes . it ' s always different . [FP] the talking they don ' t work . yeah . [SEP]
n/a,(0.40),WERNICKE,0.37,[CLS] very good . yes . it ' s always different . [FP] the talking they don ' t work . yeah . [SEP]


----------------------------------------------------------------------
True:  WERNICKE Pred:  ANOMIC
Text:  but [FP] there hasn't [FP] words.


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
n/a,(0.68),ANOMIC,0.88,[CLS] but [FP] there hasn ' t [FP] words . [SEP]
n/a,(0.27),WERNICKE,-1.83,[CLS] but [FP] there hasn ' t [FP] words . [SEP]


----------------------------------------------------------------------


# Important event scenario interpretation
Three examples for conduction and three for control (# of examples defined by n)

In [11]:
data = pd.DataFrame(columns=['source_file','scenario','preprocessed_text','label'])

for x in model.config.label2id.keys():
    data = pd.concat([data, texts[(texts["label"] == x) & (texts["scenario"] == scenarios[1])].sample(n=n)])

In [12]:
interpret(data)

True:  ANOMIC Pred:  ANOMIC
Text:  and [FP] in october fifty nine of nineteen sixty. [FP] in fact it's gonna be fifty years [FP] for next for this october. so i really cherish that. oh yes.


----------------------------------------------------------------------
True:  ANOMIC Pred:  ANOMIC
Text:  do this. this. and then i'd kinda have a little. and of course it got to be one or two stuff. so i finally about [FP] i think i drive about [FP] nine years ago like that. you know i went out with and did that. went to the thing. and it was all harmful [FP] harmless i thought. you know [FP] [FP] it it's gets stressful when it. i had four sets of babies you know. i had this little girl right here now. then there there was one and a third. the the first one it was three years old before this [FP] my firstname. then she comes in. and then i of course extended to to two the [FP] [FP] five five four more time was the baby yeah. it was [FP] [FP]. it was a lot because we did a college [FP] high school. then we had our junior high. we had our third grade. and you was having maybe [FP] [FP] kindergarten is starting you know. so i was with every every. so [FP] and they were good kids and all 

----------------------------------------------------------------------
True:  ANOMIC Pred:  ANOMIC
Text:  most events were a blur in that i hadta bring my children up by myself. and there was lots of problems with that. [FP] i went back to school. and i i spent most of my time bringing up my children. i mean that was the focus. that was i got graduated. i got my degrees and everything but my whole focus was on the kids. my [FP] i kept i just kept i kept going. my kids went to school. i i finally got a degree. not a degree. [FP] in the education at that point i started by then i started working on my career. i mean i worked on education but i had it was [FP] more involved on my correction career. [FP] and that's what i did until i fell apart. until that. i mean and it was the last month or two last say two or three months [FP] two months i started falling apart. [FP] and part of it now i know was [FP] the not the legs so much but there was something wrong and i knew it. obviously it was

----------------------------------------------------------------------
True:  WERNICKE Pred:  ANOMIC
Text:  okay. [FP] sure [FP] [FP] let's see. that is complex. and i i will [FP] i i'll tell you. [FP] firstnamem, my girl, [FP] she was i think ten. [FP] firstnamem. and firstnamee is eight. and the difference is [FP] very [FP] it's like night and day. [FP] i'm healing. and a lot of things i sit in my room not knowing anything. so firstnamee realize. okay i'm gonna step it up. and she [FP] my wife she is also doing that. but it's clear. she he needs me. me. so [FP] there is a shift. okay? the opposite [FP] firstnamee is clueless now. and he she sits in her room. what to do? and she says not knowing anything. i'll do art, painting. and [FP] so in retrospect [FP] it made sense because she [FP] she there is noone. i'm doing nothing. and [FP] firstnamee is doing something. now i realize now like three years and i i says. oh she's got to do more! so i did [FP] [FP] what's they call it [FP] ta

----------------------------------------------------------------------
True:  WERNICKE Pred:  WERNICKE
Text:  you want a good time? sad. oh to have my wife. have her. and it last [FP]. that's how long. and we were very happy. no no no no no no. it was four five seven eight eight. no to four feet. four. i could see her. bitter. only at timing. it's only short. yeah. two. no. one two three. yes. [FP] to be here. right rare. we get married. oh yeah. first one, long time. and then then i found the other one, took the new one. and i go with her. and it's her also lost. and then he came with the other. so that's why he came together. where [FP] we worked. yeah in the tear. in the air force.


----------------------------------------------------------------------
True:  WERNICKE Pred:  ANOMIC
Text:  it could be anything. i like. i like driving the car. [FP] i like going on driving. i i like anything. i bought i i i do anything. i [FP] will do anything. [FP] it's weird to say. i will do any. mhm. i got married. i got [FP] [FP] [FP] i heard a lot of things. it it's a quench. which one you want? oh ours was it's just like because our ours was just a separate ceremony. her. it's me and her. it's us. mhm.


----------------------------------------------------------------------


# Cinderella scenario interpretation
Three examples for conduction and three for control (# of examples defined by n)

In [13]:
data = pd.DataFrame(columns=['source_file','scenario','preprocessed_text','label'])

for x in model.config.label2id.keys():
    data = pd.concat([data, texts[(texts["label"] == x) & (texts["scenario"] == scenarios[2])].sample(n=n)])

In [14]:
interpret(data)

True:  ANOMIC Pred:  ANOMIC
Text:  and the [FP] [FP] stepsisters and [FP] stepmom is is [FP] not friendly not friendly. so how cinderella [FP] cinderella hope the [FP] the ball dance? ball dance? because [FP] [FP] this [FP] this is [UP] torn and tattered. and the wish her dream does. wish [FP] a ballroom bar. so the there's [FP] a fairy. and it wage a wand. the cinderella's a beautiful dress and [FP] class skippers [FP] [FP] class glass glass slippers. and so there's going to the the ball. and cinderella had [FP] a man. he's a handsome man. that's the prince. so [FP] [FP] [FP] there's [FP] the the fairy said. [UP] [FP] two three ten eleven twelve. twelve o'clock and that's it! there's that's it. you got it. so [FP] the cinderella said. i forgot the twelve o'clock. so oh she. i gotta go. i gotta go. so he left. and the [FP] the prince [FP] the prince of cinderella was infatuated [FP] the prince. [FP] tit for tat. so the prince he said. there there is [FP] a notice. hear ye hear ye. you 

----------------------------------------------------------------------
True:  ANOMIC Pred:  ANOMIC
Text:  and [FP] she lived in a home. [FP] and the [FP] there were two or three other [FP] young ladies about the same age. [FP] and she were [FP] they were need [FP] [FP] they're nasty people. they weren't nice. she hadta work all the work. she did everything a hard. and [FP] you know she was somewhat alone in that regard. [FP] so [FP] [FP] eventually [FP] [FP] [FP] she kind of [FP] became in love with [FP] a prince. [FP] she liked that guy. and [FP] they were hoping that [FP] maybe they might [FP] some way when they get a little bit older they might get married [FP] because they loved each other. and what happened is [FP] she she lost [FP] [FP] the prince a little bit. what is her name? [FP] they became [FP] very got married eventually. so i think what the story i think. my sense of it. yeah.


----------------------------------------------------------------------
True:  WERNICKE Pred:  ANOMIC
Text:  and [FP] she's happy. [FP] sad but [FP] in good spirits. the girl is her orphanage. and [FP] [FP] what's the word? his mom. and there is a ball. and everything [FP] the the girls. oh a ball! wow! so then that is good. the cinderella is not coming. [FP] she is working, cleaning up all that. and also there is they birds, mice. and all cleaning up also. so cinderella is she is going. so that's good. [FP] [FP] so that it was perfect. and even the prince had arrived. and he is spitting. okay? [FP] the ball is over. and i wanna there is a second time. [FP] so [FP] second time. and also nice with the cinderella and the man the the prince. and also it's nice. now but [FP] there is a [FP] godmother. and he is a she is a a special powers. and [FP] that's good. but [FP] there's a limit with [FP] midnight. so [FP] she is sad. and she goes. now there's a slipper. she forgot it. or i don't kno

----------------------------------------------------------------------
True:  WERNICKE Pred:  WERNICKE
Text:  oh she wants to be a queen. but you know. and. mother. says that it can't go on. and then he goes. goes. gets. leads because of and runs in round. means of robes. he and finds working and. and he good. if if that makes sense. i don't know. they went to the mall. they want to met. they want to. they dress. they left. left. left. he left it with her during with the. so so it was lot in the ruby yeah.


----------------------------------------------------------------------


# Stroke scenario interpretation
Three examples for conduction and three for control (# of examples defined by n)

In [15]:
data = pd.DataFrame(columns=['source_file','scenario','preprocessed_text','label'])

for x in model.config.label2id.keys():
    df = texts[(texts["label"] == x) & (texts["scenario"] == scenarios[3])]
    # no controls in stroke scenario
    if len(df) >= n:
        data = pd.concat([data, texts[(texts["label"] == x) & (texts["scenario"] == scenarios[3])].sample(n=n)])

In [16]:
interpret(data)

True:  ANOMIC Pred:  ANOMIC
Text:  i remember remember yes mhm. i went around the block. drove a a bicycle. went around the block. came home. got off my bicycle. and then i sat down and and and i told my wife. call a doctor. i'm i'm out of my mind you know. it's not right you know. so she called me. [FP] he called. and [FP] they took me to the hospital. and about fifteen days' worth something like that. and [FP] gave me an operation. [FP] and then i came home you know [FP]. and we played the ball. basketball [FP] football. not football but balls and stuff like that. and i got better. and then i came out to janice whiteside's place. and [FP] about ten years eleven years ago. at least eleven oh or twelve. one and one on one. and [FP] and [FP] we just you know talked about the [UP] things i learned. and i could drive a car. i mean you know stuff like that's no problem but [FP] this talking to someone that was a problem you know. yeah it was very problem a real problem. [FP].


----------------------------------------------------------------------
True:  ANOMIC Pred:  ANOMIC
Text:  because i can't pronounce words. and i can't remember words. yes. all everything? i just got up from the computer. i stood up. this side went numb. and then fell down. and then crawled up to other room. fell down. made it up and fell down four times. and then [FP] i fod called my friend. and [FP] he as soon soon as he saw me he knew what was happened. i didn't know what's happened to me. yes. just that i know was i couldn't do this. oh yeah. that's a soon as he saw me he called the ambulance. what do you mean by that? yeah yeah. oh and i went and then to a nursing home in [FP] [UP] not in where i stayed for two months or three months. and then i get home finally. yes! yes. i mean when i when i [FP] had the stroke i was. and i couldn't speak. i couldn't walk. at seven thirty in the morning. yes.


----------------------------------------------------------------------
True:  ANOMIC Pred:  ANOMIC
Text:  oh we [FP] i had a stroke. and i was [FP] bed laid up about [FP] three or four or five days. and then i had some [FP] on the the the. but [FP] i [FP] and that's it. yes now for the recovery i have had about one two three four five six operations. [FP]. and a couple. and i did that by in about oh then twelve then for wait fifteen weeks weeks ago. and then [FP] i i i [FP] that and that. and i did the that. and i did i did a little bit of the that. but i could had [FP] [FP]. i could have lots of time. i did the homework and [FP] that for texas tech for about twelve to one fifteen months ago. and i had them. and and i or this had they have a a a system where i did [FP] a [FP] a [FP] stuck stroke [FP] a a a head heading or something. [FP] and i sksstk. and i have had about three three three or two deals with them. and i had [FP] good some they said i said. [FP] and i did [FP] extra extr

----------------------------------------------------------------------
True:  WERNICKE Pred:  WERNICKE
Text:  sometime in in yeah [FP] around in there sometime. i woke up and i was gonna go with my sister and all with somebody. and i didn't feel too too hot. so i called her and told her to come and tell me i didn't feel good. and she told me to took me to her normal little little place and they didn't even look at me. they took me automatically to the store to the car or to the hospital. and i don't know what they. and that's where i know. i haven't. i haven't. not in my book! everybody says that i'm always getting better but i don't feel like i am. i can't stand to do this junk stair by myself. i've fallen three times in a row the very same day. i just don't wanna be around it that much. and i do it here because i hafta and when i get home i'm in my my chair.


----------------------------------------------------------------------
True:  WERNICKE Pred:  WERNICKE
Text:  because i was helping at the church. they don't get what stop the ladies. i said. don't touch the money. put it leave it to give it later. because i didn't know it was that. i thought we'll just relax. it was raining thursday night. they were doing a meeting. and then i went back there. go and i help them to be ready for the the benefit in march. in march. yeah. i let them i hired a policeman. mhm. so i got the police. before him everything was ready. i tell her. go ahead. we finish. go ahead. but i'm not well. so we not gonna in the the universe just the way pull it over the. then i come home. i say. well maybe rest and lie down to rest. and that was it. she hadta call the police. not much. i'm okay. and a [FP] [FP] was a thursday. say i can leave. that was the [FP] the federal hospital.


----------------------------------------------------------------------


# Cat scenario interpretation
Three examples for conduction and three for control (# of examples defined by n)

In [17]:
data = pd.DataFrame(columns=['source_file','scenario','preprocessed_text','label'])

for x in model.config.label2id.keys():
    data = pd.concat([data, texts[(texts["label"] == x) & (texts["scenario"] == scenarios[4])].sample(n=n)])

In [18]:
interpret(data)

True:  ANOMIC Pred:  ANOMIC
Text:  this one's a tough one. a a a. not a dog. cat. trying to race cat down [FP] down the down [FP] wants him come down. with the and he he wants to wants to rode his rode his bike bicycle. he share a bicycle over to get get his cat. [FP] the young man is up the tree being chased by a by a dog. [FP] keeping him keep him up the thing. and a a bird watching singing. [FP] with the [FP] two gentleman with a [FP] ice going over to the thing to either either bring bring the gentleman up with the dog or the girl with the [FP] with the [FP] dog no cat. and and watching coming over. and ladder labor day. that's one of them i can't get. i can't figure it out. i know i know what is it. i got one at home. go up and down! ladder! it's a ladder. finally. and a ladder. and they brought the ladder over to bring up either the dog or get the dog away to keep the man or save the save the kitten with the with the with the daughter.


----------------------------------------------------------------------
True:  ANOMIC Pred:  ANOMIC
Text:  a girl [FP] [FP] looks at cat. and a fire engine is coming. and the dog is barking. and a man no catch cat because [FP] i think [FP] because when he get there it breaks and he falls down. so firemen come with a ladder [FP] save him.


----------------------------------------------------------------------
True:  ANOMIC Pred:  ANOMIC
Text:  he [FP] first one. he is he wanna get cat here. and this here this this one got fall down. [FP] well first one. she's going to to help help help it cat going get here. second one here. third one here. i don't. [FP] that's first one last one he got filers [FP] filer one here. hold his laddert.


----------------------------------------------------------------------
True:  WERNICKE Pred:  ANOMIC
Text:  okay. okay. okay. but then girl. [FP] well crawling [FP] cat. yeah cat. [FP] [FP] then girl [FP] then well father maybe up in the tree getting the cat but not reaching the the cat. [FP] then [FP] [FP] well not [FP] bicycle [FP] on the ground. ladder. oh dog [FP] oh [FP] [FP] [FP] dog [FP] [FP] on the ground. then a fire [FP] trier no fire truck arrived. then that [FP] cat i think so that cat and firemen were truck arrive. then that ladder [FP] that cat [FP] now well now [FP] that cat [FP] then going [FP] ladder finally coming down. i think so. but [FP] [FP] fire truck [FP] go. [FP] [FP] ladder. yeah. yeah yeah. yeah yeah.


----------------------------------------------------------------------
True:  WERNICKE Pred:  ANOMIC
Text:  and the firemen the firemen firemen. an he's good get toes. so he's in [FP] in. so [FP] i don't know. he had.


----------------------------------------------------------------------
True:  WERNICKE Pred:  ANOMIC
Text:  we've had this one. yeah. yeah. okay [FP] for me i think the whole the whole thing about it is it's [FP] a man. he has a daughter. and the man had got up into his tree. [FP] and there's a cat like a cat there. and the man. i don't. up in the tree. he can't get the cat. he can't get himself. there's a dog. it's on the on the ground yelling [FP] screaming. not screaming. but he's he's barking at him, at the the man. [FP] so there's a little bit of a problem. and then the firemen came. those are great. so they were gonna try and get the man off of this and probably get the dog [FP] the cat out of this off of this also. oh and the little. did i say this? the the little. yeah it i think the little.


----------------------------------------------------------------------
